In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

C:\Users\JoonHo Lee\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
input_data

In [2]:
# 학습 파라미터
learning_rate = 0.001
training_steps = 20000
batch_size = 128
display_step = 200

In [3]:
# 모델에 대한 파라미터 
num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

In [4]:
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

weight = tf.Variable(tf.random_normal([num_hidden, num_classes]))
biases = tf.Variable(tf.random_normal([num_classes]))

In [5]:
def RNN(X, weight, biases):
    X = tf.unstack(X, timesteps, 1)
    
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, X, dtype=tf.float32)
    out = tf.matmul(outputs[-1], weight) + biases
    return out

In [6]:
logits = RNN(X, weight, biases)
prediction = tf.nn.softmax(logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [8]:
batch_x

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
# Start Training
with tf.Session() as sess:
    sess.run(init)
    
    # Training Dataset
    for step in range(1, training_steps + 1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # shape가 (batch_size, 728)로 되어있다. 
        batch_x = batch_x.reshape((batch_size, timesteps, num_input)) # (batch_size, 28, 28)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        # Display Training Status
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Step "+str(step)+", Minibatch Loss= "+"{:.4f}".format(loss) + \
                  ", Training Accuracy= "+"{:.3f}".format(acc))
    print("Opimtization Finished")
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.6470, Training Accuracy= 0.117
Step 200, Minibatch Loss= 2.1597, Training Accuracy= 0.211
Step 400, Minibatch Loss= 1.9286, Training Accuracy= 0.367
Step 600, Minibatch Loss= 1.8407, Training Accuracy= 0.359
Step 800, Minibatch Loss= 1.7378, Training Accuracy= 0.414
Step 1000, Minibatch Loss= 1.6621, Training Accuracy= 0.492
Step 1200, Minibatch Loss= 1.4988, Training Accuracy= 0.539
Step 1400, Minibatch Loss= 1.5554, Training Accuracy= 0.477
Step 1600, Minibatch Loss= 1.3988, Training Accuracy= 0.594
Step 1800, Minibatch Loss= 1.3936, Training Accuracy= 0.492
Step 2000, Minibatch Loss= 1.3602, Training Accuracy= 0.555
Step 2200, Minibatch Loss= 1.2634, Training Accuracy= 0.570
Step 2400, Minibatch Loss= 1.1302, Training Accuracy= 0.602
Step 2600, Minibatch Loss= 1.0875, Training Accuracy= 0.688
Step 2800, Minibatch Loss= 1.1291, Training Accuracy= 0.688
Step 3000, Minibatch Loss= 1.0553, Training Accuracy= 0.586
Step 3200, Minibatch Loss= 1.1152, Training Acc

KeyboardInterrupt: 